Ноутбук с экспериментами по построению convex hull по синтетическим данным модели 3.

In [11]:
import pickle
import sys
import numpy as np
import pandas as pd
import artm
import seaborn as sns
import matplotlib.pyplot as plt
print artm.version()

from os import path, mkdir
from datetime import datetime
sys.path.insert(0, '..\\modules\\helpers')

import distances_helper as dh 
import print_helper as ph
import create_model_helper as cmh
import build_convex_hull_helper as bchh
import test_vs_original_columns_helper as tvsoch
import compare_two_models as ctm

from plot_helper import PlotMaker
from config_helper import ConfigPaths
from scipy.optimize import minimize

0.8.1


In [3]:
config = ConfigPaths('config_sample_m3.cfg')
print config.models_file_name
models_file = open(config.models_file_name, 'a')

plot_maker = PlotMaker()

batch_vectorizer = artm.BatchVectorizer(data_path=config.output_batches_path,
                                        data_format='batches')
dictionary = artm.Dictionary()
dictionary.load(dictionary_path=config.dictionary_path + '.dict')

Q:\\topic_modeling\\csi_science_collections.git\experiments\pn_model3\np_27_03_oe\models.txt


In [20]:
def load_pickle_file(filename, _path=config.experiment_path):
    pickle_filename = path.join(_path, filename)
    pickle_file = open(pickle_filename, 'rb')
    p_file = pickle.load(pickle_file)
    pickle_file.close()
    return p_file

Загрузим оригинальный sample датасет (от model3), до этого скопировав в папку с batches нужные pickle файлы модели.

In [5]:
phi_original, theta_original = load_pickle_file('phi_model3.p', config.output_batches_path), load_pickle_file('theta_model3.p', config.output_batches_path)
print phi_original.shape, theta_original.shape

(2216, 100) (100, 3446)


# Построить выпуклую оболочку по матрице
Создадим матрицу phi_Test, в/о которой будем строить

In [17]:
def create_model_1(n_iteration, n_topics=200):
    model_name = 'm1_iter_{}'.format(n_iteration)
    tmp_model = cmh.create_model(current_dictionary=dictionary, n_topics=n_topics, n_doc_passes=5, seed_value=100 + n_iteration,
                                 n_top_tokens=15, p_mass_threshold=0.25)
    tmp_model = cmh.fit_one_model(plot_maker, batch_vectorizer, models_file, config, 
                                  tmp_model, _n_iterations=25, 
                                  _model_name=model_name)
    return tmp_model
tmp_model = create_model_1(0)
phi_test = tmp_model.get_phi()

name = m1_iter_0, n_topics = 200, n_doc_passes = 5, seed_value = 100, n_iterations = 25, top_tokens_score = 15, topic_kernel_score = 0.25



Строим в/о с помощью решения оптимизационной задачи (get_em_result_one_matrix).
Каждый раз фильтруем по порогу (opt_fun_threshold) не одну л/з тему, а сразу несколько (для ускорения), но так, чтобы они были далеко друг от друга (n_closest_topics_count).

In [19]:
get_topics_to_remove_fn = bchh.get_topics_to_remove_by_opt_fun_and_distance
dist_fn = dh.hellinger_dist
n_closest_topics_count = 15
get_result_one_matrix_fn = bchh.get_optimization_result_one_matrix # get_em_result_one_matrix
opt_fun_threshold = 0.6

phi_convex_hull, iterations_info_filter = bchh.filter_convex_hull(phi_test, get_topics_to_remove_fn, dist_fn, 
                                                                     get_result_one_matrix_fn,
                                                                     n_closest_topics_count, opt_fun_threshold, max_iteration=50,
                                                                     previous_iterations_info_list=[],
                                                                     use_previous_iterations=False)

[2017-03-27 20:28:28.449000] filtering iteration = 1 / 50
Column topic_17 not optimized
Column topic_124 not optimized
Column topic_141 not optimized
Column topic_194 not optimized
[2017-03-27 20:29:18.887000] 14 topics to remove, 50 not_removed_topics_count because close topics, current convex_hull shape = (2216, 186)
[2017-03-27 20:29:18.887000] filtering iteration = 2 / 50
Column topic_50 not optimized
Column topic_64 not optimized
[2017-03-27 20:30:06.172000] 9 topics to remove, 27 not_removed_topics_count because close topics, current convex_hull shape = (2216, 177)
[2017-03-27 20:30:06.172000] filtering iteration = 3 / 50
[2017-03-27 20:30:50.429000] 4 topics to remove, 19 not_removed_topics_count because close topics, current convex_hull shape = (2216, 173)
[2017-03-27 20:30:50.429000] filtering iteration = 4 / 50
Column topic_63 not optimized
Column topic_141 not optimized
[2017-03-27 20:31:34.645000] 6 topics to remove, 10 not_removed_topics_count because close topics, current

Спроектируем phi_convex_hull на phi_original 

In [21]:
distances_to_original = dh.calculate_distances(dist_fn, phi_convex_hull, phi_original)
opt_res_to_original = bchh.get_optimization_result(dist_fn, None, phi_convex_hull, phi_original, distances_to_original,
                                                   n_closest_topics=n_closest_topics_count)

Column topic_100 not optimized


Посчитаем число колонок из phi_convex_hull, которые удовл. условию - расстояние от них до л/к из оригинальных столбцов < порога. 

In [30]:
th = 0.4
close_cols_count = [opt_col_name for opt_col_name, val in opt_res_to_original.iteritems() if val['fun'] < th]
print 'th =', th, 'close_cols_count =', len(close_cols_count)

th = 0.4 close_cols_count = 85
